<a href="https://colab.research.google.com/github/motahari64/MedBot---Medical-Chatbot/blob/main/medbot_main2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain openai tiktoken unstructured chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.2 MB

In [ ]:
import os
#Specify your API key in OPENAI
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/




**Build a vectorDB**

In [ ]:
# Build a sample vectorDB
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

#Specify the path to the folder containing the TXT files
path = ''
loader = DirectoryLoader(path, glob="**/*.txt")
data=loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
len(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


50

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
## Now we need to create a tool for our retriever. The main things we need to pass in are a name for the retriever as well as a description.
## These will both be used by the language model, so they should be informative.

from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever,
   "search_documents",
    "Searches and returns documents regarding the txt file in resource folder",
)

tools = [tool]

In [ ]:
question_input = input("what is your question ? ")

what is youe question ? where is my dad


**Entity Extraction**

In [ ]:
from langchain.chains import create_extraction_chain
from langchain.chat_models import ChatOpenAI

# Schema
schema = {
    "properties": {
        "sickness": {"type": "string"},
        "drug": {"type": "string"},
        "symptoms": {"type": "string"},
        "symptoms_organ": {"type": "string"},
        "symptoms_extra_info": {"type": "string"}
    },
    "required": ["sickness","drug","symptoms"],
}

# Input
inp = question_input
# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)

extract=chain.run(inp)


chain.run(inp)         ##show all



[]

**Agent Constructor**

In [ ]:
## Here, we will use the high level create_conversational_retrieval_agent API to construct the agent.

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain.prompts import PromptTemplate



prompt_template = PromptTemplate.from_template(
    "answer the question this : {question} by search only available resource text. Extract the answer to the question '{question}' or say 'not found' if the information is not contained."
)
if(extract):
  agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)
  result = agent_executor(
    {
        "input":prompt_template.format(question=question_input)
    }
  )
else : print("I am medical assistant, please ask your medical question only")

result["output"]


I am medical assistant,please ask your medical question only


'Acetaminophen is mentioned in multiple available resource texts. Here are some excerpts:\n\n1. From the article "ear-infection-te6203.txt":\n   - "Give your child an over-the-counter pain reliever like acetaminophen (such as Tylenol) or ibuprofen (such as Advil or Motrin)."\n   - "If you give medicine to your baby, follow your doctor\'s advice about what amount to give."\n   - "Do not give aspirin to anyone younger than 20. Aspirin has been linked to Reye syndrome, a serious illness."\n\n2. From the article "cancer-pain-ncicdr0000062738.txt":\n   - "Step 1 treats mild pain. Patients in this category receive nonopioid analgesics such as acetaminophen, nonsteroidal anti-inflammatory drugs, or an adjuvant analgesic, if necessary."\n   - "Step 2 agents include tramadol and acetaminophen products containing hydrocodone, oxycodone, and codeine."\n\nThese are just a few examples of the mentions of acetaminophen in the available resource texts.'